# Modélisation de la température autour d'un groupe de manchots

### 1) Présentation du problème

Les manchots se regroupent pour se tenir chaud. 
Nous allons étudier l'influence de la direction, de la norme du vent et du nombre de manchots sur l'évolution de la température extérieure autour du groupe de manchots. 

Le domaine étudié est un domaine carré $\Omega$=ABCD avec A, B, C et D les sommets du carré, autour du groupe de manchots. L'intérieur du groupe de manchots est considéré comme étant extérieur au domaine. On appelle la frontière entre notre domaine et le groupe de manchots $ \partial \Omega_{int} $
On suppose que l'on observe le groupe de manchots vu de haut.

La température à l'extérieur du groupe de manchots suit l'équation d'advection diffusion : 
$\overrightarrow{w}\nabla(T) = D\Delta(T) $

Avec $\overrightarrow{w}$ le vent   
D = $\frac{\lambda}{c_P \rho} $ le coefficient de diffusion thermique  
T : la température extérieure au groupe de manchots  

A T = 250 K, on a D = 1.6E-6 $s.m^{-2}$
       
On utilise des conditions au bord de Dirichlet avec :  
    - T = 39°C, la température des manchots, donc aux bords du domaines autour du groupe de manchots.  
    - T = -23 °C, la température de l'air loin du groupe du manchots, donc sur les bords extérieurs du domaine.

### 2) Mise en équations du problème

On étudie le problème suivant : 
    $\left \{
        \begin{array}{lll}
            \overrightarrow{w}\nabla(T) = D\Delta(T)  dans \Omega \\
            \left \{
                \begin{array}{ll}
                    T = Tp & \mbox{si}  \ x   \in \partial \Omega_{int}  \\ 
                    T = T_{\inf} & \mbox{sur}  \  [A,B] \cup [B,C] \cup [C,D] \cup [D,A]  \\
                \end{array}
            \right.
        \end{array}
    \right.  $
    
On simule le problème pour n pingouins
On considère comme longueur caractéristique le rayon qu'aurait un disque occupé par ce nombre de pingouins.
On adimensionne notre problème en posant :   

$ T* = \frac{T-T_{\inf}}{T-T_{p}} \\
\overrightarrow{w*} = \frac{\overrightarrow{w}}{w0} $ avec w0 la vitesse du vent


$ Pe =  \frac{w0 L}{D} $ le nombre de Péclet.

Le problème adimensionné devient : 
$\left \{
        \begin{array}{lll}
            \overrightarrow{w*}\nabla(T*) = Pe\Delta(T*) \  dans \  \Omega \\
            \left \{
                \begin{array}{ll}
                    T = 1 & \mbox{si}  \ x   \in \partial \Omega_{int}  \\ 
                    T = 0 & \mbox{sur}  \  [A,B] \cup [B,C] \cup [C,D] \cup [D,A]  \\
                \end{array}
            \right.
        \end{array}
    \right.  $
    
Par souci de simplification, on omet les * dans les notations. 

La formulation variationnelle devient

"Trouver $ u \in H=\{v\in H^{1}(\Omega)\,:\,v|_{[A,B] \cup [B,C] \cup [C,D] \cup [D,A]}=0\, \ et \  v|_{\partial \Omega_{int} = 0}\} $ tel que

$$\forall v\in H,\quad \int_{\Omega}(\nabla u \nabla v) = Pe \int_{\Omega}(\overrightarrow{w} \nabla T v) "$$




### 3) Résolution numérique

#### Importations

In [1]:
from dolfin import *
from fenics import *
from math import *
from mshr import *
import numpy as np
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

Le nombre de manchots est aléatoire. 

Le nombre de manchots n suit une loi de Poisson de paramètre 20.
Les manchots sont serrés les uns aux autres pour minimiser les pertes de chaleur. On assimile leur placement à un quadrillage de forme hexagonale, où les manchots sont placés aux sommets. On suppose que tous les points à l'intérieur du groupe sont occupés. La frontière du groupe de manchots est uniquement déterminée par les manchots qui ont moins de six voisins. On suppose que tous les manchots du groupe ont au moins deux voisins.

On commence par générer un quadrillage avec sept manchots formant le premier hexagone (avec un manchot au centre). 
Pour faciliter la construction de la frontière du domaine, on place les manchots suivants pour compléter les hexagones au fur et à mesure.

La forme du groupe de manchot dépend donc du nombre de manchots tiré aléatoirement. La géométrie du domaine varie donc de manière aléatoire.

Une fois le groupe formé, on suppose que le nombre de manchots présents dans le groupe reste constant. 

### Tirage du nombre de manchots

In [ ]:
poisson = 20
n = np.random.poisson(poisson)

### Placement des manchots

In [ ]:
b = sqrt(3)/2

centres = [0+0j, 1.5+b*1j, 0+2*b*1j, -1.5+b*1j, -1.5-b*1j, 0-2*b*1j, 1.5-b*1j]

premiers_pingouins = [0+0j, 1+0j,0.5+b*1j ,-0.5+b*1j,-1+0j, -0.5-b*1j, 0.5-b*1j]


Liste_pingouins_suivants = []
Liste_pingouins_exte = []
for i in range(1, len(centres)):
    depart = (i-2)
    Liste_pingouins_suivants.append(centres[i])
    for k in range(3):
        pingouin = premiers_pingouins[1:][(k+depart)%6]
        pingouin+=centres[i]
        Liste_pingouins_suivants.append(pingouin)
        Liste_pingouins_exte.append(pingouin)


def hexagone(n):
    if n <= 7: return 0
    return (n-8)//4+1

def pingouins_exte(n):
    n_hexagone = hexagone(n)
    return premiers_pingouins[n_hexagone+1:]+Liste_pingouins_exte[:n-7]

### Affichage du placement des pingouins

In [ ]:
Liste_pingouins_finale = premiers_pingouins +  Liste_pingouins_suivants
Liste_pingouins_reel = [z.real for z in Liste_pingouins_finale]
Liste_pingouins_img = [z.imag for z in Liste_pingouins_finale]

plt.plot(Liste_pingouins_reel, Liste_pingouins_img,'o', color="black")
plt.xlim([-15,15])
plt.ylim([-15,15])
plt.show()

### Expression du vent

On tire la vitesse du vent et sa composante horizontale suivant une loi normale.

In [4]:
w0_moy =  5
w0_var = 0.5
alpha_moy = 5
alpha_var = 0.5

w0 = np.random.normal(w0_moy,w0_var)
alpha = np.random.normal(alpha_moy,alpha_var)

print(w0, alpha)

def W():
    return Expression(("cos(alpha)","sin(alpha)"))

NameError: name 'np' is not defined

In [ ]:
print(W)

### Définition des paramètres

In [5]:
D = 1.6E-6 
Tinf = 250
Tp = 312
R = sqrt(n)/2

Pe= w0*R/D

NameError: name 'sqrt' is not defined

### Création du maillage

In [3]:
x_dernier_pingouin, y_dernier_pingouin = Liste_pingouins_finale[:n][-1].real, Liste_pingouins_finale[:n][-1].imag
if abs(x_dernier_pingouin) <= abs(y_dernier_pingouin):
    x_coupure = x_dernier_pingouin
    if y_dernier_pingouin <= 0 : 
        y_coupure = -15
    else : 
        y_coupure = 15
else : 
    y_coupure = y_dernier_pingouin
    if x_dernier_pingouin <= 0 : 
        x_coupure = -15
    else : 
        x_coupure = 15
    
pingouins_vertices = [Point(pingouin.real, pingouin.imag) for pingouin in pingouins_exte(n)]
domain_vertices = [(x_coupure, y_coupure), (15,-15), (15, 15), (-15, 15), (-15,-15)]+pingouins_vertices
domain = Polygon(domain_vertices)
mesh = generate_mesh(domain,40)

NameError: name 'Liste_pingouins_finale' is not defined

In [ ]:
Hh2 = FunctionSpace(mesh, 'P', 2)

### Conditions de bord

In [ ]:
def liste_pingouins(n):
    return Liste_pingouins_finale[:n]

def test_dedans(n, xp, yp):
    L_triangles = []
    L_pingouins = liste_pingouins(n)
    for i in range(len(L_pingouins)):
        for j in range(i+1, len(L_pingouins)):
            for k in range(j+1, len(L_pingouins)):
                if abs(L_pingouins[i]-L_pingouins[j])< 1.2 and abs(L_pingouins[i]-L_pingouins[k])< 1.2 and abs(L_pingouins[k]-L_pingouins[j])< 1.2:
                    L_triangles.append((L_pingouins[i],L_pingouins[j],L_pingouins[k]))
    
    for triangle in L_triangles:
        x1, y1 = triangle[0].real, triangle[0].imag
        x2, y2 = triangle[1].real, triangle[1].imag
        x3, y3 = triangle[2].real, triangle[2].imag
        c1 = (x2 - x1) * (yp - y1) - (y2 - y1) * (xp - x1)
        c2 = (x3 - x2) * (yp - y2) - (y3 - y2) * (xp - x2)
        c3 = (x1 - x3) * (yp - y3) - (y1 - y3) * (xp - x3)
        if (c1 < 0 and c2 < 0 and c3 < 0) or (c1 > 0 and c2 > 0 and c3 > 0):
            return False

    return True

In [ ]:
tol = 1E-14

class Frontiere(SubDomain):
    def inside(self, x, on_boundary):
        frontiere = on_boundary
        for i in range(len(Liste_pingouins_exte)-1):
            x_1, y_1 = Liste_pingouins_exte[i].real, Liste_pingouins_exte[i].imag
            x_2, y_2 = Liste_pingouins_exte[i+1].real, Liste_pingouins_exte[i+1].imag
            ymin = min([y_1, y_2])
            ymax = max([y_1, y_2])
            if x_1 != x_2:
                a = (y_2 - y_1)/(x_2 - x_1)
                b = y_2 - a*x_2
                frontiere = frontiere or (abs(x[1] - a*x[0]+b) < tol ) and  (x[1]<ymax and x[1]>ymin)
            if x_1 == x_2:            
                frontiere = frontiere or (( abs(x[0]-x_1)<tol) and (x[1]<ymax and x[1]>ymin))

        return frontiere
        

class Cadre_Top(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and (abs(x[0]-15)<tol) 
    
class Cadre_Bot(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and (abs(x[0]+15)<tol) 

class Cadre_Right(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and (abs(x[1]-15)<tol) 
    
class Cadre_Left(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and (abs(x[0]+15)<tol) 

 
boundaries = MeshFunction("size_t", mesh, mesh.topology().dim()-1, 0)
boundaries_bool = MeshFunction("bool", mesh, mesh.topology().dim()-1, 0)
#boundaries_double = MeshFunction("double", mesh, mesh.topology().dim()-1, 0)


boundaries.set_all(5)
boundaries_bool.set_all(True)
#boundaries_double.set_all(0.3)

frontiere = Frontiere()
frontiere.mark(boundaries, 0)
frontiere.mark(boundaries_bool, False)

cadre_top = Cadre_Top()
cadre_bot = Cadre_Bot()
cadre_right = Cadre_Right()
cadre_left = Cadre_Left()

cadre_top.mark(boundaries, 1)
cadre_left.mark(boundaries, 2)
cadre_bot.mark(boundaries, 3)
cadre_right.mark(boundaries, 4)

ds = Measure("ds", domain=mesh, subdomain_data=boundaries_bool)

DirichCond = [DirichletBC(Hh2, Constant(1), frontiere), DirichletBC(Hh2, Constant(Tinf), cadre_top), DirichletBC(Hh2, Constant(Tinf), cadre_bot), DirichletBC(Hh2, Constant(Tinf), cadre_right), DirichletBC(Hh2, Constant(Tinf), cadre_left)]

### Résolution de la formulation variationnelle

In [ ]:
u = TrialFunction(Hh2)
v = TestFunction(Hh2)
F = (inner(grad(u), grad(v))-Pe*(cos(alpha)*grad(u)[0]+sin(alpha)*grad(u)[1]*v))*dx


a = lhs(F)
l = rhs(F)

u = Function(Hh2)
solve(a == l, u, DirichCond)

### Tracé de la solution

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

x= np.linspace(-15, 15, 1)
y = np.linspace(-15, 15, 1)
ax.plot(x, y)

x0 = 0
y0 = -10
xw = w0*cos(alpha)
yw = w0*sin(alpha)

arr = plt.Arrow(x0, y0, xw, yw, edgecolor='black')
ax.add_patch(arr)
arr.set_facecolor('b')



plu = plot(u)
plt.title('Champ de température autour des pingouins')
plt.xlabel('x')
plt.ylabel('y')
plt.colorbar(plu)

plt.show()

### 4) Conclusion 

La forme du groupe de manchots ainsi que la direction du vent et son intensité ont donc une influence importante sur la température autour de ce groupe de manchots.

Sources utilisées : 

http://www.breves-de-maths.fr/la-marche-de-lempereur/

https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0050277
    
https://en.wikipedia.org/wiki/Convection%E2%80%93diffusion_equation
https://fr.wikipedia.org/wiki/Air
    
https://fenicsproject.org/olddocs/dolfin/1.3.0/python/demo/documented/poisson/python/documentation.html

https://fenicsproject.org/docs/dolfin/2016.2.0/python/demo/documented/subdomains-poisson/python/documentation.html